# $\alpha$=0.1

In [7]:
import torch
import torchvision.transforms as transforms              
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torchvision.models import Inception_V3_Weights
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import Subset
from src.temperature_scaling import ModelWithTemperature
from src.aps import split_data_set, aps_scores, aps_classification, eval_aps

# load pre-trained model InceptionV3 and set mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1).to(device)

#  Reprocess: Center Crop and then resize to 299*299
data_transform = transforms.Compose([
    transforms.CenterCrop(299),
    transforms.Resize(299), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

sorted_val_path = "D:\\Download\\ImageNet-1K\\Validation_Set\\sorted_ImageNet_val"
dataset = ImageFolder(root=sorted_val_path, transform=data_transform)

# Temperature Scaling
model.eval() # only use output.logits of Inception's output
subset_size = len(dataset) // 10
indices = np.random.choice(len(dataset), subset_size, replace=False)
subset_dataset = Subset(dataset, indices)
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False, num_workers=4)

model_with_temp = ModelWithTemperature(model, temperature = 1.0).to(device)
model_with_temp.set_temperature(train_loader)
model_with_temp.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("\n")
print("APS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False,num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Before temperature - NLL: 1.110, ECE: 0.023
Optimal temperature: 0.969
After temperature - NLL: 1.102, ECE: 0.028


APS Classification, Start!

Running experiment 1/10...
q_hat = 0.8723027646541596
Total set size: 1358792
Total coverage sets: 22497
Total samples amount: 25000
Average Prediction Set Size After APS in runs 1: 54.35168
Average Coverage Rate in runs 1: 89.99%

Running experiment 2/10...
q_hat = 0.8728827476501465
Total set size: 1337350
Total coverage sets: 22374
Total samples amount: 25000
Average Prediction Set Size After APS in runs 2: 53.494
Average Coverage Rate in runs 2: 89.50%

Running experiment 3/10...
q_hat = 0.876809561252594
Total set size: 1411422
Total coverage sets: 22634
Total samples amount: 25000
Average Prediction Set Size After APS in runs 3: 56.45688
Average Coverage Rate in runs 3: 90.54%

Running experiment 4/10...
q_hat = 0.8713245928287507
Total set size: 1357670
Total coverage sets: 22484
Total samples amount: 25000
Average Prediction Set Size Af

# $\alpha$=0.2

In [1]:
# error rate
alpha = 0.2

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("\n")
print("APS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False,num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Before temperature - NLL: 1.012, ECE: 0.021
Optimal temperature: 0.960
After temperature - NLL: 0.999, ECE: 0.025


APS Classification, Start!

Running experiment 1/10...
q_hat = 0.772050428390503
Total set size: 445006
Total coverage sets: 20057
Total samples amount: 25000
Average Prediction Set Size After APS in runs 1: 17.80024
Average Coverage Rate in runs 1: 80.23%

Running experiment 2/10...
q_hat = 0.7680832982063294
Total set size: 413027
Total coverage sets: 19843
Total samples amount: 25000
Average Prediction Set Size After APS in runs 2: 16.52108
Average Coverage Rate in runs 2: 79.37%

Running experiment 3/10...
q_hat = 0.7743975281715393
Total set size: 444087
Total coverage sets: 20089
Total samples amount: 25000
Average Prediction Set Size After APS in runs 3: 17.76348
Average Coverage Rate in runs 3: 80.36%

Running experiment 4/10...
q_hat = 0.7709043860435486
Total set size: 446222
Total coverage sets: 19972
Total samples amount: 25000
Average Prediction Set Size Afte

# $\alpha$=0.05

In [2]:
# error rate
alpha = 0.05

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("\n")
print("APS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False,num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")



APS Classification, Start!

Running experiment 1/10...
q_hat = 0.9284281939268111
Total set size: 3075276
Total coverage sets: 23758
Total samples amount: 25000
Average Prediction Set Size After APS in runs 1: 123.01104
Average Coverage Rate in runs 1: 95.03%

Running experiment 2/10...
q_hat = 0.9260983943939208
Total set size: 2930914
Total coverage sets: 23637
Total samples amount: 25000
Average Prediction Set Size After APS in runs 2: 117.23656
Average Coverage Rate in runs 2: 94.55%

Running experiment 3/10...
q_hat = 0.9312667518854141
Total set size: 3205492
Total coverage sets: 23886
Total samples amount: 25000
Average Prediction Set Size After APS in runs 3: 128.21968
Average Coverage Rate in runs 3: 95.54%

Running experiment 4/10...
q_hat = 0.926798340678215
Total set size: 3021469
Total coverage sets: 23741
Total samples amount: 25000
Average Prediction Set Size After APS in runs 4: 120.85876
Average Coverage Rate in runs 4: 94.96%

Running experiment 5/10...
q_hat = 0.92

# Result  
$\alpha$=0.1  
From the above test, following results can be collected :
- Final Average Prediction Set Size: 54.45
- Final Average Coverage: 89.96%  

$\alpha$=0.2  
From the above test, following results can be collected :
- Final Average Prediction Set Size: 17.63
- Final Average Coverage: 80.12%  

$\alpha$=0.05  
From the above test, following results can be collected :
- Final Average Prediction Set Size: 120.98
- Final Average Coverage: 94.98%